In [1]:
"""
Sistem de Testare pe Bază de Cicluri (Cycle-Based Testing) pentru Autovehicule
Testează vehiculele prin cicluri realiste de pornire, accelerare, frânare, schimbări de trepte
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Tuple
from enum import Enum
import json
from pathlib import Path

plt.style.use('seaborn-v0_8-whitegrid' if 'seaborn-v0_8-whitegrid' in plt.style.available else 'default')
plt.rcParams['figure.figsize'] = (14, 8)


class CycleType(Enum):
    """Tipuri de cicluri de testare"""
    COLD_START = "Pornire la rece"
    HOT_START = "Pornire la cald"
    ACCELERATION = "Accelerare"
    BRAKING = "Frânare"
    GEAR_SHIFT = "Schimbare treaptă"
    IDLE = "Ralanti"
    CRUISE = "Mers constant"
    EMERGENCY_BRAKE = "Frânare de urgență"


class DrivingProfile(Enum):
    """Profile de conducere"""
    CITY = "Urban"
    HIGHWAY = "Autostradă"
    MIXED = "Mixt"
    AGGRESSIVE = "Agresiv"
    ECO = "Economic"
    SPORT = "Sportiv"


class SystemStress(Enum):
    """Niveluri de stres pentru sisteme"""
    LOW = 1
    MEDIUM = 2
    HIGH = 3
    EXTREME = 4


@dataclass
class CycleDefinition:
    """Definiție ciclu de testare"""
    cycle_type: CycleType
    duration: float  # secunde
    stress_level: SystemStress
    speed_profile: List[float]  # km/h
    rpm_profile: List[float]  # RPM motor
    gear: Optional[int] = None
    brake_force: Optional[float] = None  # 0-100%
    throttle: Optional[float] = None  # 0-100%


@dataclass
class CycleResult:
    """Rezultat execuție ciclu"""
    cycle_number: int
    cycle_type: CycleType
    start_time: float
    duration: float
    success: bool
    defects_detected: List[str] = field(default_factory=list)
    system_degradation: Dict[str, float] = field(default_factory=dict)
    max_temperature: float = 0.0
    avg_rpm: float = 0.0
    fuel_consumed: float = 0.0


class VehicleComponent:
    """Componentă vehicul cu degradare pe bază de cicluri"""
    
    def __init__(self, name: str, max_cycles: int, wear_per_cycle: float):
        self.name = name
        self.max_cycles = max_cycles
        self.wear_per_cycle = wear_per_cycle
        self.current_cycles = 0
        self.condition = 100.0
        self.failures = []
        
    def execute_cycle(self, stress_level: SystemStress) -> Tuple[bool, Optional[str]]:
        """
        Execută un ciclu pe componentă
        
        Returns:
        --------
        Tuple[bool, Optional[str]] : (success, defect_message)
        """
        self.current_cycles += 1
        
        # Uzură bazată pe stres
        stress_multiplier = stress_level.value
        wear = self.wear_per_cycle * stress_multiplier
        
        # Uzură accelerată după 70% din cicluri
        usage_ratio = self.current_cycles / self.max_cycles
        if usage_ratio > 0.7:
            wear *= (1 + (usage_ratio - 0.7) * 2)
        
        self.condition -= wear
        self.condition = max(0, self.condition)
        
        # Probabilitate defect
        failure_prob = 0.001 * stress_multiplier * (1 - self.condition / 100)
        
        if np.random.random() < failure_prob or self.condition < 10:
            defect_msg = f"{self.name}: Defect la ciclu {self.current_cycles} (stare: {self.condition:.1f}%)"
            self.failures.append(defect_msg)
            return False, defect_msg
        
        return True, None


class CycleBasedTester:
    """Sistem principal de testare pe bază de cicluri"""
    
    def __init__(self, vehicle_id: str):
        self.vehicle_id = vehicle_id
        self.test_start = datetime.now()
        
        # Inițializare componente cu cicluri maxime realiste
        self.components = {
            'Motor': VehicleComponent('Motor', max_cycles=500000, wear_per_cycle=0.00002),
            'Electromotor': VehicleComponent('Electromotor', max_cycles=100000, wear_per_cycle=0.0001),
            'Ambreiaj': VehicleComponent('Ambreiaj', max_cycles=150000, wear_per_cycle=0.00007),
            'Cutie viteze': VehicleComponent('Cutie viteze', max_cycles=200000, wear_per_cycle=0.00005),
            'Sistem frânare': VehicleComponent('Sistem frânare', max_cycles=50000, wear_per_cycle=0.0002),
            'Plăcuțe frână': VehicleComponent('Plăcuțe frână', max_cycles=30000, wear_per_cycle=0.00033),
            'Suspensie': VehicleComponent('Suspensie', max_cycles=100000, wear_per_cycle=0.0001),
            'Baterie': VehicleComponent('Baterie', max_cycles=50000, wear_per_cycle=0.0002),
            'Alternator': VehicleComponent('Alternator', max_cycles=80000, wear_per_cycle=0.000125)
        }
        
        self.cycle_results = []
        self.total_cycles_executed = 0
        self.test_log = []
        
    def create_driving_profile(self, profile_type: DrivingProfile, 
                              num_cycles: int) -> List[CycleDefinition]:
        """
        Creează un profil complet de conducere cu cicluri
        
        Parameters:
        -----------
        profile_type : DrivingProfile
            Tipul profilului de conducere
        num_cycles : int
            Numărul total de cicluri
        
        Returns:
        --------
        List[CycleDefinition] : Lista ciclurilor
        """
        cycles = []
        
        if profile_type == DrivingProfile.CITY:
            # Profil urban: multe porniri, frânări, viteze mici
            pattern = [
                (CycleType.COLD_START, 1),
                (CycleType.ACCELERATION, 5),
                (CycleType.CRUISE, 3),
                (CycleType.BRAKING, 5),
                (CycleType.IDLE, 2),
                (CycleType.GEAR_SHIFT, 8),
            ]
            
        elif profile_type == DrivingProfile.HIGHWAY:
            # Profil autostradă: puține cicluri, viteză constantă
            pattern = [
                (CycleType.HOT_START, 1),
                (CycleType.ACCELERATION, 2),
                (CycleType.CRUISE, 10),
                (CycleType.BRAKING, 1),
                (CycleType.GEAR_SHIFT, 3),
            ]
            
        elif profile_type == DrivingProfile.AGGRESSIVE:
            # Conducere agresivă: accelerări și frânări bruște
            pattern = [
                (CycleType.HOT_START, 1),
                (CycleType.ACCELERATION, 8),
                (CycleType.BRAKING, 7),
                (CycleType.EMERGENCY_BRAKE, 2),
                (CycleType.GEAR_SHIFT, 12),
            ]
            
        elif profile_type == DrivingProfile.ECO:
            # Conducere economică: tranziții lente
            pattern = [
                (CycleType.COLD_START, 1),
                (CycleType.ACCELERATION, 3),
                (CycleType.CRUISE, 8),
                (CycleType.BRAKING, 2),
                (CycleType.GEAR_SHIFT, 4),
            ]
            
        elif profile_type == DrivingProfile.SPORT:
            # Conducere sportivă: turații mari
            pattern = [
                (CycleType.HOT_START, 1),
                (CycleType.ACCELERATION, 6),
                (CycleType.CRUISE, 4),
                (CycleType.BRAKING, 5),
                (CycleType.GEAR_SHIFT, 10),
            ]
            
        else:  # MIXED
            # Profil mixt
            pattern = [
                (CycleType.COLD_START, 1),
                (CycleType.HOT_START, 2),
                (CycleType.ACCELERATION, 5),
                (CycleType.CRUISE, 6),
                (CycleType.BRAKING, 4),
                (CycleType.GEAR_SHIFT, 8),
                (CycleType.IDLE, 2),
            ]
        
        # Generare cicluri pe baza pattern-ului
        cycles_generated = 0
        while cycles_generated < num_cycles:
            for cycle_type, weight in pattern:
                if cycles_generated >= num_cycles:
                    break
                
                for _ in range(weight):
                    if cycles_generated >= num_cycles:
                        break
                    
                    cycle = self._create_cycle(cycle_type, profile_type)
                    cycles.append(cycle)
                    cycles_generated += 1
        
        return cycles[:num_cycles]
    
    def _create_cycle(self, cycle_type: CycleType, 
                     profile: DrivingProfile) -> CycleDefinition:
        """Creează un ciclu individual cu parametri realiști"""
        
        if cycle_type == CycleType.COLD_START:
            return CycleDefinition(
                cycle_type=cycle_type,
                duration=5.0,
                stress_level=SystemStress.HIGH,
                speed_profile=[0, 0],
                rpm_profile=[0, 800, 1200, 900],
                gear=None,
                throttle=30.0
            )
        
        elif cycle_type == CycleType.HOT_START:
            return CycleDefinition(
                cycle_type=cycle_type,
                duration=2.0,
                stress_level=SystemStress.MEDIUM,
                speed_profile=[0, 0],
                rpm_profile=[0, 800, 900],
                gear=None,
                throttle=20.0
            )
        
        elif cycle_type == CycleType.ACCELERATION:
            if profile == DrivingProfile.AGGRESSIVE:
                stress = SystemStress.EXTREME
                throttle = 90.0
                rpm_max = 6000
                speed_end = 120
            elif profile == DrivingProfile.ECO:
                stress = SystemStress.LOW
                throttle = 40.0
                rpm_max = 2500
                speed_end = 60
            else:
                stress = SystemStress.MEDIUM
                throttle = 60.0
                rpm_max = 4000
                speed_end = 80
            
            return CycleDefinition(
                cycle_type=cycle_type,
                duration=8.0,
                stress_level=stress,
                speed_profile=[0, speed_end * 0.3, speed_end * 0.6, speed_end],
                rpm_profile=[800, rpm_max * 0.6, rpm_max, rpm_max * 0.5],
                throttle=throttle
            )
        
        elif cycle_type == CycleType.BRAKING:
            if profile == DrivingProfile.AGGRESSIVE:
                stress = SystemStress.HIGH
                brake_force = 85.0
            else:
                stress = SystemStress.MEDIUM
                brake_force = 50.0
            
            return CycleDefinition(
                cycle_type=cycle_type,
                duration=4.0,
                stress_level=stress,
                speed_profile=[80, 40, 10, 0],
                rpm_profile=[2500, 1500, 800, 800],
                brake_force=brake_force
            )
        
        elif cycle_type == CycleType.EMERGENCY_BRAKE:
            return CycleDefinition(
                cycle_type=cycle_type,
                duration=2.5,
                stress_level=SystemStress.EXTREME,
                speed_profile=[100, 40, 0],
                rpm_profile=[3000, 800, 800],
                brake_force=100.0
            )
        
        elif cycle_type == CycleType.GEAR_SHIFT:
            return CycleDefinition(
                cycle_type=cycle_type,
                duration=1.5,
                stress_level=SystemStress.MEDIUM,
                speed_profile=[40, 50],
                rpm_profile=[2500, 1800, 2200],
                gear=np.random.randint(2, 5)
            )
        
        elif cycle_type == CycleType.CRUISE:
            speed = 90 if profile == DrivingProfile.HIGHWAY else 50
            return CycleDefinition(
                cycle_type=cycle_type,
                duration=30.0,
                stress_level=SystemStress.LOW,
                speed_profile=[speed, speed],
                rpm_profile=[2000, 2000],
                throttle=30.0
            )
        
        else:  # IDLE
            return CycleDefinition(
                cycle_type=cycle_type,
                duration=20.0,
                stress_level=SystemStress.LOW,
                speed_profile=[0, 0],
                rpm_profile=[800, 800],
                throttle=0.0
            )
    
    def execute_cycle(self, cycle: CycleDefinition, 
                     cycle_number: int) -> CycleResult:
        """
        Execută un ciclu și înregistrează rezultatele
        
        Parameters:
        -----------
        cycle : CycleDefinition
            Definiția ciclului de executat
        cycle_number : int
            Numărul ciclului
        
        Returns:
        --------
        CycleResult : Rezultatul execuției
        """
        start_time = self.total_cycles_executed * 0.1  # Timestamp simplificat
        defects = []
        degradation = {}
        
        # Componente afectate de fiecare tip de ciclu
        affected_components = {
            CycleType.COLD_START: ['Motor', 'Electromotor', 'Baterie'],
            CycleType.HOT_START: ['Motor', 'Electromotor'],
            CycleType.ACCELERATION: ['Motor', 'Cutie viteze', 'Ambreiaj'],
            CycleType.BRAKING: ['Sistem frânare', 'Plăcuțe frână'],
            CycleType.EMERGENCY_BRAKE: ['Sistem frânare', 'Plăcuțe frână', 'Suspensie'],
            CycleType.GEAR_SHIFT: ['Ambreiaj', 'Cutie viteze'],
            CycleType.CRUISE: ['Motor', 'Alternator'],
            CycleType.IDLE: ['Motor', 'Baterie']
        }
        
        components = affected_components.get(cycle.cycle_type, ['Motor'])
        success = True
        
        # Execută ciclul pe fiecare componentă afectată
        for component_name in components:
            component = self.components[component_name]
            comp_success, defect_msg = component.execute_cycle(cycle.stress_level)
            
            degradation[component_name] = component.condition
            
            if not comp_success:
                defects.append(defect_msg)
                success = False
        
        # Calcul parametri cicluri
        max_temp = 90 + cycle.stress_level.value * 10 + np.random.normal(0, 5)
        avg_rpm = np.mean(cycle.rpm_profile)
        fuel = cycle.duration * (avg_rpm / 1000) * 0.01  # Consum simplificat
        
        result = CycleResult(
            cycle_number=cycle_number,
            cycle_type=cycle.cycle_type,
            start_time=start_time,
            duration=cycle.duration,
            success=success,
            defects_detected=defects,
            system_degradation=degradation,
            max_temperature=max_temp,
            avg_rpm=avg_rpm,
            fuel_consumed=fuel
        )
        
        self.cycle_results.append(result)
        self.total_cycles_executed += 1
        
        return result
    
    def run_test_plan(self, profile_type: DrivingProfile, 
                     num_cycles: int,
                     stop_on_failure: bool = True,
                     verbose: bool = True) -> Dict:
        """
        Rulează un plan complet de testare
        
        Parameters:
        -----------
        profile_type : DrivingProfile
            Tipul profilului de conducere
        num_cycles : int
            Numărul de cicluri de executat
        stop_on_failure : bool
            Oprește testul la prima defecțiune
        verbose : bool
            Afișează progres detaliat
        
        Returns:
        --------
        Dict : Rezultatele complete ale testului
        """
        print("=" * 80)
        print(f"START TEST PE BAZĂ DE CICLURI: {self.vehicle_id}")
        print("=" * 80)
        print(f"Profil conducere: {profile_type.value}")
        print(f"Cicluri planificate: {num_cycles:,}")
        print(f"Oprire la defecțiune: {'Da' if stop_on_failure else 'Nu'}")
        print()
        
        # Creare plan cicluri
        cycles = self.create_driving_profile(profile_type, num_cycles)
        
        if verbose:
            # Statistici plan
            cycle_counts = {}
            for cycle in cycles:
                cycle_counts[cycle.cycle_type] = cycle_counts.get(cycle.cycle_type, 0) + 1
            
            print("Distribuție cicluri planificate:")
            for cycle_type, count in sorted(cycle_counts.items(), key=lambda x: x[1], reverse=True):
                pct = (count / len(cycles)) * 100
                print(f"  {cycle_type.value:25s}: {count:6,} ({pct:5.1f}%)")
            print()
        
        # Execuție cicluri
        test_stopped = False
        stop_reason = "Toate ciclurile executate cu succes"
        
        for i, cycle in enumerate(cycles, 1):
            if verbose and i % 100 == 0:
                print(f"Progres: {i:,}/{num_cycles:,} cicluri ({i/num_cycles*100:.1f}%)")
            
            result = self.execute_cycle(cycle, i)
            
            if not result.success:
                if verbose:
                    print(f"\n⚠️  Ciclu {i}: {cycle.cycle_type.value}")
                    for defect in result.defects_detected:
                        print(f"    ❌ {defect}")
                
                if stop_on_failure:
                    stop_reason = f"Defecțiune la ciclu {i}: {result.defects_detected[0]}"
                    test_stopped = True
                    print(f"\n🛑 TEST OPRIT: {stop_reason}")
                    break
        
        # Generare rezultate finale
        results = self._generate_results(stop_reason, test_stopped)
        
        print("\n" + "=" * 80)
        print("TEST FINALIZAT")
        print("=" * 80)
        print(f"Cicluri executate: {self.total_cycles_executed:,}/{num_cycles:,}")
        print(f"Cicluri reușite: {sum(1 for r in self.cycle_results if r.success):,}")
        print(f"Defecțiuni: {sum(len(r.defects_detected) for r in self.cycle_results):,}")
        
        return results
    
    def _generate_results(self, stop_reason: str, stopped: bool) -> Dict:
        """Generează raport complet rezultate"""
        
        # Statistici cicluri
        cycle_stats = {}
        for cycle_type in CycleType:
            type_results = [r for r in self.cycle_results if r.cycle_type == cycle_type]
            if type_results:
                cycle_stats[cycle_type] = {
                    'total': len(type_results),
                    'success': sum(1 for r in type_results if r.success),
                    'failed': sum(1 for r in type_results if not r.success),
                    'avg_temp': np.mean([r.max_temperature for r in type_results]),
                    'total_fuel': sum(r.fuel_consumed for r in type_results)
                }
        
        # Starea componentelor
        component_status = {}
        for name, component in self.components.items():
            component_status[name] = {
                'condition': component.condition,
                'cycles_completed': component.current_cycles,
                'max_cycles': component.max_cycles,
                'usage_percent': (component.current_cycles / component.max_cycles) * 100,
                'failures': len(component.failures)
            }
        
        # Defecțiuni totale
        all_defects = []
        for result in self.cycle_results:
            for defect in result.defects_detected:
                all_defects.append({
                    'cycle_number': result.cycle_number,
                    'cycle_type': result.cycle_type.value,
                    'message': defect
                })
        
        results = {
            'vehicle_id': self.vehicle_id,
            'test_stopped': stopped,
            'stop_reason': stop_reason,
            'total_cycles_executed': self.total_cycles_executed,
            'total_cycles_planned': len(self.cycle_results),
            'successful_cycles': sum(1 for r in self.cycle_results if r.success),
            'failed_cycles': sum(1 for r in self.cycle_results if not r.success),
            'cycle_statistics': cycle_stats,
            'component_status': component_status,
            'all_defects': all_defects,
            'total_fuel_consumed': sum(r.fuel_consumed for r in self.cycle_results),
            'avg_temperature': np.mean([r.max_temperature for r in self.cycle_results])
        }
        
        return results
    
    def generate_report(self, results: Dict, output_dir: str = 'cycle_test_results'):
        """Generează raport complet cu vizualizări"""
        
        print("\n" + "=" * 80)
        print("GENERARE RAPORT")
        print("=" * 80)
        
        Path(output_dir).mkdir(exist_ok=True)
        
        # 1. Raport text
        self._generate_text_report(results, output_dir)
        
        # 2. Excel
        self._generate_excel_report(results, output_dir)
        
        # 3. Vizualizări
        self._generate_visualizations(results, output_dir)
        
        # 4. JSON
        self._save_json(results, output_dir)
        
        print(f"\n✓ Raport complet: {output_dir}/")
    
    def _generate_text_report(self, results: Dict, output_dir: str):
        """Generează raport text"""
        
        report_path = f"{output_dir}/raport_cicluri_{self.vehicle_id}.txt"
        
        with open(report_path, 'w', encoding='utf-8') as f:
            f.write("=" * 80 + "\n")
            f.write(f"RAPORT TEST PE BAZĂ DE CICLURI\n")
            f.write(f"Vehicul: {results['vehicle_id']}\n")
            f.write(f"Data: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write("=" * 80 + "\n\n")
            
            # Rezumat
            f.write("REZUMAT EXECUȚIE\n")
            f.write("-" * 80 + "\n")
            f.write(f"Cicluri executate: {results['total_cycles_executed']:,}\n")
            f.write(f"Cicluri reușite: {results['successful_cycles']:,}\n")
            f.write(f"Cicluri eșuate: {results['failed_cycles']:,}\n")
            f.write(f"Rata succes: {results['successful_cycles']/results['total_cycles_executed']*100:.2f}%\n")
            f.write(f"Test oprit prematur: {'Da' if results['test_stopped'] else 'Nu'}\n")
            if results['test_stopped']:
                f.write(f"Motiv: {results['stop_reason']}\n")
            f.write(f"Combustibil total: {results['total_fuel_consumed']:.2f} L\n")
            f.write(f"Temperatură medie: {results['avg_temperature']:.1f}°C\n\n")
            
            # Statistici pe tip ciclu
            f.write("STATISTICI PER TIP CICLU\n")
            f.write("-" * 80 + "\n")
            for cycle_type, stats in results['cycle_statistics'].items():
                f.write(f"\n{cycle_type.value}:\n")
                f.write(f"  Total: {stats['total']:,}\n")
                f.write(f"  Reușite: {stats['success']:,}\n")
                f.write(f"  Eșuate: {stats['failed']:,}\n")
                f.write(f"  Temp medie: {stats['avg_temp']:.1f}°C\n")
                f.write(f"  Combustibil: {stats['total_fuel']:.2f} L\n")
            
            # Starea componentelor
            f.write("\n\nSTARE COMPONENTE\n")
            f.write("-" * 80 + "\n")
            for name, status in sorted(results['component_status'].items(), 
                                      key=lambda x: x[1]['condition']):
                f.write(f"\n{name}:\n")
                f.write(f"  Stare: {status['condition']:.1f}%\n")
                f.write(f"  Cicluri: {status['cycles_completed']:,}/{status['max_cycles']:,} " +
                       f"({status['usage_percent']:.1f}%)\n")
                f.write(f"  Defecțiuni: {status['failures']}\n")
            
            # Defecțiuni
            if results['all_defects']:
                f.write("\n\nDEFECȚIUNI DETECTATE\n")
                f.write("-" * 80 + "\n")
                for i, defect in enumerate(results['all_defects'], 1):
                    f.write(f"\n{i}. Ciclu {defect['cycle_number']} ({defect['cycle_type']})\n")
                    f.write(f"   {defect['message']}\n")
        
        print(f"✓ Raport text: {report_path}")
    
    def _generate_excel_report(self, results: Dict, output_dir: str):
        """Generează raport Excel"""
        
        excel_path = f"{output_dir}/raport_cicluri_{self.vehicle_id}.xlsx"
        
        with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
            # Sheet 1: Rezumat
            summary_data = {
                'Parametru': [
                    'Vehicul ID',
                    'Cicluri executate',
                    'Cicluri reușite',
                    'Cicluri eșuate',
                    'Rata succes (%)',
                    'Combustibil total (L)',
                    'Temperatură medie (°C)',
                    'Test oprit'
                ],
                'Valoare': [
                    results['vehicle_id'],
                    results['total_cycles_executed'],
                    results['successful_cycles'],
                    results['failed_cycles'],
                    f"{results['successful_cycles']/results['total_cycles_executed']*100:.2f}",
                    f"{results['total_fuel_consumed']:.2f}",
                    f"{results['avg_temperature']:.1f}",
                    'Da' if results['test_stopped'] else 'Nu'
                ]
            }
            pd.DataFrame(summary_data).to_excel(writer, sheet_name='Rezumat', index=False)
            
            # Sheet 2: Stare componente
            comp_data = []
            for name, status in results['component_status'].items():
                comp_data.append({
                    'Componentă': name,
                    'Stare (%)': status['condition'],
                    'Cicluri': status['cycles_completed'],
                    'Max cicluri': status['max_cycles'],
                    'Utilizare (%)': status['usage_percent'],
                    'Defecțiuni': status['failures']
                })
            pd.DataFrame(comp_data).to_excel(writer, sheet_name='Componente', index=False)
            
            # Sheet 3: Statistici cicluri
            cycle_stats_data = []
            for cycle_type, stats in results['cycle_statistics'].items():
                cycle_stats_data.append({
                    'Tip ciclu': cycle_type.value,
                    'Total': stats['total'],
                    'Reușite': stats['success'],
                    'Eșuate': stats['failed'],
                    'Temp medie (°C)': f"{stats['avg_temp']:.1f}",
                    'Combustibil (L)': f"{stats['total_fuel']:.2f}"
                })
            pd.DataFrame(cycle_stats_data).to_excel(writer, sheet_name='Statistici cicluri', index=False)
            
            # Sheet 4: Defecțiuni
            if results['all_defects']:
                defects_df = pd.DataFrame(results['all_defects'])
                defects_df.to_excel(writer, sheet_name='Defecțiuni', index=False)
        
        print(f"✓ Raport Excel: {excel_path}")
    
    def _generate_visualizations(self, results: Dict, output_dir: str):
        """Generează grafice"""
        
        print("\nGenerare grafice...")
        
        # 1. Evoluția stării componentelor
        fig, ax = plt.subplots(figsize=(14, 8))
        
        conditions_over_time = {name: [] for name in self.components.keys()}
        cycle_numbers = []
        
        for i, result in enumerate(self.cycle_results):
            cycle_numbers.append(result.cycle_number)
            for name in self.components.keys():
                if name in result.system_degradation:
                    conditions_over_time[name].append(result.system_degradation[name])
                elif conditions_over_time[name]:
                    conditions_over_time[name].append(conditions_over_time[name][-1])
                else:
                    conditions_over_time[name].append(100)
        
        for name, conditions in conditions_over_time.items():
            if conditions:
                ax.plot(cycle_numbers[:len(conditions)], conditions, label=name, linewidth=2, alpha=0.7)
        
        ax.set_xlabel('Număr ciclu', fontsize=12)
        ax.set_ylabel('Stare componentă (%)', fontsize=12)
        ax.set_title('Evoluția Stării Componentelor pe Cicluri', fontsize=14, fontweight='bold')
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        ax.grid(alpha=0.3)
        plt.tight_layout()
        plt.savefig(f'{output_dir}/01_evolutie_componente.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("  ✓ 01_evolutie_componente.png")
        
        # 2. Distribuție cicluri
        fig, ax = plt.subplots(figsize=(12, 8))
        
        cycle_types = []
        cycle_counts = []
        colors_list = []
        
        for cycle_type, stats in sorted(results['cycle_statistics'].items(), 
                                       key=lambda x: x[1]['total'], reverse=True):
            cycle_types.append(cycle_type.value)
            cycle_counts.append(stats['total'])
            # Culoare bazată pe rata de succes
            success_rate = stats['success'] / stats['total'] if stats['total'] > 0 else 1
            colors_list.append('#1a9850' if success_rate > 0.95 else '#fdae61' if success_rate > 0.8 else '#d73027')
        
        bars = ax.barh(cycle_types, cycle_counts, color=colors_list)
        ax.set_xlabel('Număr cicluri', fontsize=12)
        ax.set_title('Distribuție Cicluri Executate', fontsize=14, fontweight='bold')
        ax.grid(axis='x', alpha=0.3)
        
        # Adăugare valori
        for bar in bars:
            width = bar.get_width()
            ax.text(width, bar.get_y() + bar.get_height()/2, 
                   f'{int(width):,}', ha='left', va='center', fontweight='bold')
        
        plt.tight_layout()
        plt.savefig(f'{output_dir}/02_distributie_cicluri.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("  ✓ 02_distributie_cicluri.png")
        
        # 3. Starea finală componente
        fig, ax = plt.subplots(figsize=(12, 8))
        
        components = list(results['component_status'].keys())
        conditions = [results['component_status'][c]['condition'] for c in components]
        colors = ['#d73027' if c < 30 else '#fdae61' if c < 60 else '#1a9850' for c in conditions]
        
        bars = ax.barh(components, conditions, color=colors)
        ax.axvline(x=50, color='orange', linestyle='--', label='Prag atenție', linewidth=2)
        ax.axvline(x=30, color='red', linestyle='--', label='Prag critic', linewidth=2)
        ax.set_xlabel('Stare (%)', fontsize=12)
        ax.set_title('Stare Finală Componente', fontsize=14, fontweight='bold')
        ax.legend()
        ax.grid(axis='x', alpha=0.3)
        
        for bar, cond in zip(bars, conditions):
            ax.text(cond, bar.get_y() + bar.get_height()/2, 
                   f'{cond:.1f}%', ha='left', va='center', fontweight='bold')
        
        plt.tight_layout()
        plt.savefig(f'{output_dir}/03_stare_finala.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("  ✓ 03_stare_finala.png")
        
        # 4. Utilizare componente (cicluri executate vs maxime)
        fig, ax = plt.subplots(figsize=(12, 8))
        
        comp_names = list(results['component_status'].keys())
        usage_pct = [results['component_status'][c]['usage_percent'] for c in comp_names]
        
        # Sortare după utilizare
        sorted_data = sorted(zip(comp_names, usage_pct), key=lambda x: x[1], reverse=True)
        comp_names_sorted, usage_sorted = zip(*sorted_data)
        
        colors = ['#d73027' if u > 80 else '#fdae61' if u > 50 else '#1a9850' for u in usage_sorted]
        
        bars = ax.barh(comp_names_sorted, usage_sorted, color=colors)
        ax.set_xlabel('Utilizare cicluri (%)', fontsize=12)
        ax.set_title('Utilizare Componentelor (% din cicluri maxime)', fontsize=14, fontweight='bold')
        ax.grid(axis='x', alpha=0.3)
        
        for bar, usage in zip(bars, usage_sorted):
            ax.text(usage, bar.get_y() + bar.get_height()/2, 
                   f'{usage:.1f}%', ha='left', va='center', fontweight='bold')
        
        plt.tight_layout()
        plt.savefig(f'{output_dir}/04_utilizare_componente.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("  ✓ 04_utilizare_componente.png")
    
    def _save_json(self, results: Dict, output_dir: str):
        """Salvează rezultate în JSON"""
        
        # Conversie Enum la string
        json_results = {
            'vehicle_id': results['vehicle_id'],
            'test_stopped': results['test_stopped'],
            'stop_reason': results['stop_reason'],
            'total_cycles_executed': results['total_cycles_executed'],
            'successful_cycles': results['successful_cycles'],
            'failed_cycles': results['failed_cycles'],
            'total_fuel_consumed': results['total_fuel_consumed'],
            'avg_temperature': results['avg_temperature'],
            'cycle_statistics': {
                k.value: v for k, v in results['cycle_statistics'].items()
            },
            'component_status': results['component_status'],
            'defects': results['all_defects']
        }
        
        json_path = f"{output_dir}/date_cicluri_{self.vehicle_id}.json"
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(json_results, f, indent=2, ensure_ascii=False)
        
        print(f"✓ Date JSON: {json_path}")


def main():
    """Funcția principală - exemple de utilizare"""
    
    print("\n" + "=" * 80)
    print("SISTEM DE TESTARE PE BAZĂ DE CICLURI")
    print("=" * 80)
    
    # Creare tester
    tester = CycleBasedTester("VEH-CYCLE-001")
    
    # Configurare test
    PROFILE = DrivingProfile.CITY  # Alege: CITY, HIGHWAY, MIXED, AGGRESSIVE, ECO, SPORT
    NUM_CYCLES = 5000  # Număr cicluri
    
    print(f"\nConfiguție test:")
    print(f"  Profil: {PROFILE.value}")
    print(f"  Cicluri: {NUM_CYCLES:,}")
    
    # Rulare test
    results = tester.run_test_plan(
        profile_type=PROFILE,
        num_cycles=NUM_CYCLES,
        stop_on_failure=True,  # Oprește la prima defecțiune
        verbose=True
    )
    
    # Generare raport
    tester.generate_report(results)
    
    print("\n" + "=" * 80)
    print("✓ TEST COMPLET")
    print("=" * 80)


if __name__ == "__main__":
    main()


SISTEM DE TESTARE PE BAZĂ DE CICLURI

Configuție test:
  Profil: Urban
  Cicluri: 5,000
START TEST PE BAZĂ DE CICLURI: VEH-CYCLE-001
Profil conducere: Urban
Cicluri planificate: 5,000
Oprire la defecțiune: Da

Distribuție cicluri planificate:
  Schimbare treaptă        :  1,664 ( 33.3%)
  Accelerare               :  1,045 ( 20.9%)
  Frânare                  :  1,040 ( 20.8%)
  Mers constant            :    626 ( 12.5%)
  Ralanti                  :    416 (  8.3%)
  Pornire la rece          :    209 (  4.2%)

Progres: 100/5,000 cicluri (2.0%)
Progres: 200/5,000 cicluri (4.0%)
Progres: 300/5,000 cicluri (6.0%)
Progres: 400/5,000 cicluri (8.0%)
Progres: 500/5,000 cicluri (10.0%)
Progres: 600/5,000 cicluri (12.0%)
Progres: 700/5,000 cicluri (14.0%)
Progres: 800/5,000 cicluri (16.0%)
Progres: 900/5,000 cicluri (18.0%)
Progres: 1,000/5,000 cicluri (20.0%)
Progres: 1,100/5,000 cicluri (22.0%)
Progres: 1,200/5,000 cicluri (24.0%)
Progres: 1,300/5,000 cicluri (26.0%)
Progres: 1,400/5,000 ciclu

Testare prin Cicluri Discrete:
În loc să măsoare doar "100 ore de funcționare", sistemul simulează operațiuni specifice. Profile de Conducere Realiste:
1️⃣ CITY (Urban) 🏙️
Multe porniri/opriri
Viteze mici
Ralanti frecvent
Ideal: Test uzură urbană

2️⃣ HIGHWAY (Autostradă) 🛣️
Viteză constantă mare
Puține frânări
Turații medii
Ideal: Durabilitate long-distance

3️⃣ AGGRESSIVE (Agresiv) 🏁
Accelerări bruște
Turații 6000 RPM
Frânări extreme
Ideal: Stress test

4️⃣ ECO (Economic) 🌱
Accelerări line
Turații 2500 RPM
Uzură minimă
Ideal: Baseline

5️⃣ SPORT (Sportiv) 🏎️
Turații medii-mari
Schimbări frecvente
Ideal: Performance test

6️⃣ MIXED (Mixt) 🔀
Combinație toate tipurile
Ideal: Real-world test
